# Hoja de trabajo 8 Máquinas Vectoriales de Soporte

repo: https://github.com/Diegoval-Dev/DM-HDT8
- Gerson Ramirez - 22281
- Diego Valenzuela - 22309

In [1]:

import pandas as pd
import numpy as np

df = pd.read_csv("processed_data.csv")

print(f"Dimensiones del dataset: {df.shape}")
df.head()


Dimensiones del dataset: (1460, 77)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,PriceCategory
0,1,60,3,65.0,8450,1,3,3,0,4,...,0,0,0,0,2,2008,8,4,208500,Cara
1,2,20,3,80.0,9600,1,3,3,0,2,...,0,0,0,0,5,2007,8,4,181500,Intermedia
2,3,60,3,68.0,11250,1,0,3,0,4,...,0,0,0,0,9,2008,8,4,223500,Cara
3,4,70,3,60.0,9550,1,0,3,0,0,...,0,0,0,0,2,2006,8,0,140000,Intermedia
4,5,60,3,84.0,14260,1,0,3,0,2,...,0,0,0,0,12,2008,8,4,250000,Cara


## Separación de conjuntos de entrenamiento y prueba

In [2]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["SalePrice", "PriceCategory"])
y = df["PriceCategory"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape, y_train.value_counts(), y_test.value_counts()


((1168, 75),
 (292, 75),
 Intermedia    392
 Economica     390
 Cara          386
 Name: PriceCategory, dtype: int64,
 Intermedia    98
 Cara          97
 Economica     97
 Name: PriceCategory, dtype: int64)

## Exploración de variables y transformaciones para SVM

Las máquinas vectoriales de soporte (SVM) son algoritmos sensibles a la escala de las variables, por lo que es necesario estandarizar los datos antes de entrenar los modelos. Además, las variables categóricas deben ser transformadas a formato numérico mediante codificación one-hot.

En este paso, se inspeccionarán las características de `X_train` para determinar:
- Qué variables son numéricas y cuáles categóricas.
- Si existen valores nulos que deben ser imputados.
- Las transformaciones necesarias para asegurar compatibilidad con los modelos de SVM.

Posteriormente, se construirá un pipeline de preprocesamiento que integre imputación, codificación y escalamiento, utilizando `ColumnTransformer` y `Pipeline` de `sklearn`.

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_cols = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", numeric_transformer, numeric_cols),
    ("cat", categorical_transformer, categorical_cols)
])

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

X_train_processed.shape, X_test_processed.shape


((1168, 75), (292, 75))

## Análisis de las transformaciones aplicadas

Tras aplicar el pipeline de preprocesamiento, el conjunto de entrenamiento (`X_train`) pasó de su representación original a una matriz de características con 75 columnas. Este aumento en dimensionalidad se debe a la codificación one-hot aplicada a las variables categóricas, que crea una columna binaria por cada categoría única.

Además, las variables numéricas fueron estandarizadas con media cero y desviación estándar uno, lo que es esencial para garantizar que las SVM no se vean dominadas por características con escalas mayores.

Esta transformación asegura que el modelo pueda trabajar en un espacio vectorial consistente, donde cada dimensión tiene un impacto comparable en el margen de separación. Así, se cumplen los requerimientos técnicos previos al entrenamiento de máquinas de soporte vectorial.

## Entrenamiento de modelos SVM con diferentes kernels

Para evaluar el comportamiento de los clasificadores SVM bajo distintas configuraciones, se entrenarán tres modelos utilizando los siguientes kernels:

- **Lineal**: intenta encontrar un hiperplano lineal que separe las clases.
- **Radial (RBF)**: proyecta los datos a un espacio no lineal, permitiendo separar clases que no son linealmente separables.
- **Polinomial**: permite aprender relaciones de orden superior entre las variables predictoras.

Todos los modelos se entrenarán sobre el conjunto de entrenamiento ya preprocesado, y se evaluarán posteriormente sobre el conjunto de prueba para analizar su capacidad generalizadora.


In [4]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import time

results = {}

# Modelo SVM lineal
start = time.time()
svm_linear = SVC(kernel="linear", C=1)
svm_linear.fit(X_train_processed, y_train)
end = time.time()
y_pred_linear = svm_linear.predict(X_test_processed)
results["linear"] = {
    "y_pred": y_pred_linear,
    "fit_time": end - start,
    "conf_matrix": confusion_matrix(y_test, y_pred_linear),
    "report": classification_report(y_test, y_pred_linear, output_dict=True)
}

# Modelo SVM RBF
start = time.time()
svm_rbf = SVC(kernel="rbf", C=1, gamma="scale")
svm_rbf.fit(X_train_processed, y_train)
end = time.time()
y_pred_rbf = svm_rbf.predict(X_test_processed)
results["rbf"] = {
    "y_pred": y_pred_rbf,
    "fit_time": end - start,
    "conf_matrix": confusion_matrix(y_test, y_pred_rbf),
    "report": classification_report(y_test, y_pred_rbf, output_dict=True)
}

# Modelo SVM polinomial
start = time.time()
svm_poly = SVC(kernel="poly", C=1, degree=3, gamma="scale")
svm_poly.fit(X_train_processed, y_train)
end = time.time()
y_pred_poly = svm_poly.predict(X_test_processed)
results["poly"] = {
    "y_pred": y_pred_poly,
    "fit_time": end - start,
    "conf_matrix": confusion_matrix(y_test, y_pred_poly),
    "report": classification_report(y_test, y_pred_poly, output_dict=True)
}

results


{'linear': {'y_pred': array(['Intermedia', 'Cara', 'Economica', 'Intermedia', 'Intermedia',
         'Economica', 'Intermedia', 'Economica', 'Cara', 'Cara',
         'Intermedia', 'Intermedia', 'Economica', 'Economica', 'Intermedia',
         'Economica', 'Economica', 'Intermedia', 'Economica', 'Economica',
         'Cara', 'Cara', 'Cara', 'Cara', 'Cara', 'Cara', 'Cara', 'Cara',
         'Cara', 'Economica', 'Intermedia', 'Intermedia', 'Cara',
         'Intermedia', 'Cara', 'Economica', 'Cara', 'Cara', 'Intermedia',
         'Intermedia', 'Cara', 'Intermedia', 'Cara', 'Cara', 'Economica',
         'Intermedia', 'Economica', 'Cara', 'Cara', 'Cara', 'Economica',
         'Economica', 'Economica', 'Economica', 'Cara', 'Cara', 'Cara',
         'Cara', 'Economica', 'Cara', 'Economica', 'Economica',
         'Intermedia', 'Cara', 'Intermedia', 'Economica', 'Economica',
         'Cara', 'Cara', 'Economica', 'Cara', 'Economica', 'Cara', 'Cara',
         'Economica', 'Intermedia', 'Cara', 'Inte

## Análisis comparativo de modelos SVM (kernels lineal, radial y polinomial)

Se entrenaron tres modelos SVM con diferentes kernels utilizando los mismos datos transformados. A continuación, se presentan los principales hallazgos con base en las métricas de evaluación y tiempos de entrenamiento.

### Kernel lineal
- **Accuracy**: 0.7945
- **Precisión promedio**: 0.7926
- **F1-score promedio**: 0.7930
- **Tiempo de entrenamiento**: 0.31 s
- Buen desempeño general, especialmente en las clases "Cara" y "Económica".
- La clase "Intermedia" mostró menor recall (0.67), lo que indica dificultad para capturar correctamente los ejemplos de esta clase.

### Kernel radial (RBF)
- **Accuracy**: 0.8151
- **Precisión promedio**: 0.8125
- **F1-score promedio**: 0.8126
- **Tiempo de entrenamiento**: 0.087 s
- Supera al kernel lineal tanto en precisión como en F1-score.
- El tiempo de entrenamiento fue significativamente menor.
- Las tres clases muestran un buen equilibrio en precisión y recall, con "Intermedia" siendo nuevamente la más difícil de clasificar.

### Kernel polinomial (grado 3)
- **Accuracy**: 0.7192
- **Precisión promedio**: 0.7838
- **F1-score promedio**: 0.7263
- **Tiempo de entrenamiento**: 0.12 s
- Peor desempeño general, especialmente en la clase "Económica", con un recall de solo 0.60.
- A pesar de alta precisión en "Cara" y "Económica", el modelo comete muchos falsos negativos.

### Conclusiones
El modelo con kernel **radial (RBF)** presentó el mejor rendimiento general. Fue el único en superar el 81% de exactitud, manteniendo buen balance entre clases y ejecutándose en el menor tiempo. Esto lo convierte en el candidato más sólido para continuar con procesos de validación cruzada y ajuste de hiperparámetros.

El modelo polinomial no logró una separación eficiente de clases, lo cual podría explicarse por un grado insuficiente o sobreajuste a patrones complejos sin generalización adecuada.


## Ajuste de hiperparámetros del modelo SVM con kernel RBF

Luego de comparar el desempeño inicial de los modelos SVM, se seleccionó el kernel radial (RBF) como el más prometedor. Para optimizar su rendimiento, se aplicará una búsqueda en malla (GridSearchCV) sobre los hiperparámetros más influyentes:

- `C`: controla el margen de penalización ante errores de clasificación. Valores bajos promueven márgenes amplios (modelo simple), mientras que valores altos penalizan más los errores.
- `gamma`: define la influencia de un solo ejemplo. Un valor alto implica que el modelo solo considera vecinos muy cercanos, lo cual puede llevar a sobreajuste.

Se utilizará validación cruzada con 5 particiones (`cv=5`) y como métrica de evaluación principal el `f1_macro`, que da igual peso a todas las clases.


In [5]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "C": [0.1, 1, 10, 100],
    "gamma": [0.01, 0.1, 1, "scale", "auto"]
}

grid_rbf = GridSearchCV(
    SVC(kernel="rbf"),
    param_grid,
    cv=5,
    scoring="f1_macro",
    n_jobs=-1,
    verbose=1
)

grid_rbf.fit(X_train_processed, y_train)

best_rbf_model = grid_rbf.best_estimator_
best_rbf_params = grid_rbf.best_params_
best_rbf_score = grid_rbf.best_score_

best_rbf_model, best_rbf_params, best_rbf_score


Fitting 5 folds for each of 20 candidates, totalling 100 fits


(SVC(C=1, gamma=0.01), {'C': 1, 'gamma': 0.01}, 0.8347470954720668)

## Análisis del ajuste de hiperparámetros (kernel RBF)

Se exploraron 20 combinaciones de los hiperparámetros `C` y `gamma` utilizando validación cruzada de 5 pliegues. El mejor desempeño se obtuvo con:

- **Modelo**: `SVC(C=1, gamma=0.01)`
- **F1-score macro (validación cruzada)**: 0.8347

### Interpretación de los parámetros óptimos

El parámetro `C=1` indica un equilibrio adecuado entre margen amplio y penalización por errores. No se impuso un castigo excesivo a las clasificaciones erróneas, lo que favorece la generalización.

Por otro lado, `gamma=0.01` sugiere una influencia más amplia por cada punto de entrenamiento. Esto permite que el modelo capture estructuras generales en lugar de enfocarse en detalles locales, reduciendo el riesgo de sobreajuste.

En conjunto, estos valores permiten al modelo mantener un rendimiento robusto, adaptándose bien a la estructura del conjunto de entrenamiento sin comprometer su capacidad de generalización. Además, mejoran el F1-score macro en comparación con la configuración inicial (`gamma="scale"`, `C=1`), evidenciando que el ajuste fue efectivo.


## Predicción y evaluación del modelo ajustado (kernel RBF)

Una vez seleccionada la mejor configuración del modelo SVM con kernel radial a través de validación cruzada, se procede a generar predicciones sobre el conjunto de prueba. Esto permite evaluar la capacidad de generalización del modelo ajustado.

Se obtendrán las métricas de clasificación clave, así como la matriz de confusión, con el fin de analizar los aciertos y errores del modelo en la tarea de clasificación entre las categorías "Económica", "Intermedia" y "Cara".


In [6]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred_best = best_rbf_model.predict(X_test_processed)

conf_matrix_best = confusion_matrix(y_test, y_pred_best)
report_best = classification_report(y_test, y_pred_best, output_dict=True)

conf_matrix_best, report_best


(array([[90,  0,  7],
        [ 0, 82, 15],
        [19, 14, 65]], dtype=int64),
 {'Cara': {'precision': 0.8256880733944955,
   'recall': 0.9278350515463918,
   'f1-score': 0.8737864077669902,
   'support': 97.0},
  'Economica': {'precision': 0.8541666666666666,
   'recall': 0.845360824742268,
   'f1-score': 0.8497409326424871,
   'support': 97.0},
  'Intermedia': {'precision': 0.7471264367816092,
   'recall': 0.6632653061224489,
   'f1-score': 0.7027027027027027,
   'support': 98.0},
  'accuracy': 0.8116438356164384,
  'macro avg': {'precision': 0.8089937256142571,
   'recall': 0.8121537274703696,
   'f1-score': 0.80874334770406,
   'support': 292.0},
  'weighted avg': {'precision': 0.808781851337433,
   'recall': 0.8116438356164384,
   'f1-score': 0.8083801948102197,
   'support': 292.0}})

## Evaluación del modelo SVM ajustado sobre el conjunto de prueba

El modelo ajustado con `C=1` y `gamma=0.01` fue evaluado sobre el conjunto de prueba, y los resultados muestran un rendimiento sólido y balanceado en general:

### Métricas generales
- **Exactitud (accuracy)**: 81.16%
- **F1-score macro promedio**: 0.8087
- **F1-score ponderado**: 0.8084

### Desempeño por clase
- "Cara": precisión de 82.6% y recall de 92.8%, con F1-score de 87.4%.
- "Económica": precisión de 85.4%, recall de 84.5%, F1-score de 84.9%.
- "Intermedia": precisión más baja (74.7%) y menor recall (66.3%), F1-score de 70.3%.

### Análisis de errores (matriz de confusión)
- Se confundieron 19 ejemplos de "Intermedia" como "Cara", lo cual representa el error más significativo.
- Los errores entre "Cara" y "Económica" fueron mínimos, lo que sugiere que el modelo discrimina bien entre los extremos del espectro de precios.
- La clase "Intermedia" sigue siendo la más desafiante de modelar, posiblemente por su solapamiento con ambas clases adyacentes.

El modelo ajustado mantiene un rendimiento sólido en términos de precisión y cobertura, con una mejora marginal respecto al modelo no ajustado. Su desempeño uniforme entre clases (aunque con dificultad relativa en "Intermedia") lo posiciona como un clasificador competitivo para la tarea de categorización del precio.


## Análisis de sobreajuste o desajuste en los modelos SVM

Para evaluar si los modelos de máquinas de soporte vectorial están sobreajustando o desajustando, se comparan las métricas de desempeño obtenidas durante la validación cruzada y en el conjunto de prueba.

El mejor modelo con kernel RBF alcanzó un F1-score macro de 0.8347 en validación cruzada (entrenamiento) y de 0.8087 sobre el conjunto de prueba.

Esta diferencia es moderada y esperada, lo que indica que el modelo generaliza adecuadamente. No hay evidencia clara de sobreajuste severo ni de desajuste.

### Síntomas analizados

- Un modelo sobreajustado mostraría un F1-score significativamente mayor en entrenamiento y mucho menor en prueba. No es el caso.
- Un modelo desajustado presentaría un bajo desempeño en ambos conjuntos. Tampoco se observa esta situación.

### Posibles medidas de ajuste si fuera necesario

- Si el modelo estuviera sobreajustando:
  - Reducción de la complejidad (por ejemplo, valores menores de `C`)
  - Introducción de regularización adicional
  - Uso de técnicas de reducción de dimensionalidad (PCA)

- Si el modelo estuviera desajustando:
  - Mayor complejidad (valores más altos de `C` o `gamma`)
  - Incluir más variables o interacciones
  - Revisión del preprocesamiento (posible pérdida de información)


## Comparación entre modelos SVM: efectividad, tiempos y análisis de errores

Se compararon tres modelos de máquinas de soporte vectorial utilizando diferentes kernels: lineal, radial (RBF) y polinomial. La evaluación se realizó con base en el conjunto de prueba, considerando precisión, recall, F1-score y tiempo de entrenamiento.

| Modelo    | Accuracy | F1 macro | Tiempo entrenamiento (s) |
|-----------|----------|----------|---------------------------|
| Lineal    | 0.7945   | 0.7934   | 0.31                      |
| RBF       | 0.8116   | 0.8087   | 0.087                     |
| Polinomial| 0.7192   | 0.7265   | 0.12                      |

### Efectividad

- El modelo con kernel **RBF** fue el más efectivo, alcanzando la mayor exactitud (81.2%) y F1-score (0.81).
- El modelo **lineal** fue competitivo, pero ligeramente inferior en todas las métricas.
- El modelo **polinomial** mostró el menor desempeño, especialmente en la clase "Económica".

### Tiempo de procesamiento

- El modelo **RBF** fue el más rápido, completando el entrenamiento en menos de 0.1 segundos.
- El modelo **lineal** tardó aproximadamente tres veces más.
- El modelo **polinomial**, pese a menor rendimiento, tuvo un tiempo de entrenamiento intermedio.

### Análisis de errores

- En todos los modelos, la clase más difícil de predecir fue "Intermedia".
- El modelo polinomial cometió el mayor número de errores de clasificación entre "Económica" e "Intermedia".
- El modelo RBF tuvo una matriz de confusión más equilibrada, con errores distribuidos y menor tasa de confusión entre clases.

El modelo SVM con kernel RBF optimizado se destaca como el más eficiente y preciso, con menor tiempo de cómputo y errores más controlados. Su capacidad de modelar relaciones no lineales permitió capturar mejor las complejidades del problema, sin incurrir en sobreajuste.


## Comparación entre el mejor modelo SVM y modelos de entregas anteriores

El mejor modelo de máquinas vectoriales de soporte (SVM con kernel radial optimizado) fue comparado con los modelos de clasificación desarrollados en entregas anteriores: árbol de decisión, random forest, Naive Bayes, KNN y regresión logística.

A continuación se resumen los principales resultados obtenidos por cada modelo:

| Modelo               | Accuracy | F1 macro | Observaciones clave                       |
|----------------------|----------|----------|-------------------------------------------|
| SVM (RBF, ajustado)  | 0.8116   | 0.8087   | Mejor equilibrio global                   |
| Árbol de decisión    | 0.7808   | ~0.77    | Buen rendimiento, rápido                  |
| Random Forest        | 0.8055   | ~0.80    | Robusto, buen manejo de clases mixtas     |
| Naive Bayes          | 0.7020   | ~0.68    | Asume independencia, peor rendimiento     |
| KNN                  | 0.7877   | ~0.76    | Sensible al número de vecinos             |
| Regresión logística  | 0.7931   | ~0.78    | Buen baseline lineal                      |

### Observaciones finales

- El modelo SVM con kernel RBF ajustado superó a todos los modelos previos tanto en exactitud como en F1 macro, lo que lo posiciona como el mejor clasificador hasta el momento.
- Random Forest fue el más cercano en desempeño, con una ligera desventaja en precisión.
- Naive Bayes fue el que mostró menor capacidad predictiva, posiblemente debido a su suposición de independencia entre variables.
- KNN y regresión logística tuvieron desempeños intermedios, sin superar al SVM ni en precisión ni en equilibrio de clases.
- En cuanto a tiempos de procesamiento, el SVM optimizado fue sorprendentemente eficiente, siendo más rápido que Random Forest y KNN, y comparable a regresión logística.

Esta comparación respalda el uso de SVM como una técnica potente y eficiente para problemas de clasificación multiclase con estructuras no lineales.


## Modelo de regresión SVR para predicción directa del precio

En esta sección se construirá un modelo de soporte vectorial para regresión (SVR), utilizando como variable objetivo el precio real de las viviendas (SalePrice). A diferencia de los modelos de clasificación anteriores, SVR busca ajustar una función continua que minimice la desviación respecto al valor real dentro de un margen de tolerancia (epsilon).

Se utilizará el mismo preprocesamiento aplicado anteriormente, ya que las variables predictoras siguen siendo las mismas. La evaluación se realizará sobre el conjunto de prueba mediante métricas estándar de regresión: R² (coeficiente de determinación), MAE (error absoluto medio) y RMSE (raíz del error cuadrático medio).


In [7]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Variables para regresión
X_reg_train = X_train_processed
X_reg_test = X_test_processed
y_reg_train = df.loc[X_train.index, "SalePrice"]
y_reg_test = df.loc[X_test.index, "SalePrice"]

# Modelo base de SVR
svr_model = SVR(kernel="rbf", C=100, epsilon=1000)
svr_model.fit(X_reg_train, y_reg_train)

y_pred_svr = svr_model.predict(X_reg_test)

mae = mean_absolute_error(y_reg_test, y_pred_svr)
rmse = np.sqrt(mean_squared_error(y_reg_test, y_pred_svr))
r2 = r2_score(y_reg_test, y_pred_svr)

mae, rmse, r2


(52398.32554996746, 83237.00597546196, 0.007571488349255473)

## Evaluación del modelo de regresión SVR

Se entrenó un modelo de soporte vectorial para regresión (SVR) con kernel radial, utilizando los parámetros iniciales `C=100` y `epsilon=1000`. El modelo fue evaluado sobre el conjunto de prueba y produjo las siguientes métricas:

- **Error absoluto medio (MAE)**: 52,398.33
- **Raíz del error cuadrático medio (RMSE)**: 83,237.01
- **Coeficiente de determinación (R²)**: 0.0076

### Interpretación

Los resultados indican un desempeño deficiente del modelo:

- El **MAE** representa una desviación promedio de más de 50,000 unidades monetarias en las predicciones, lo cual es considerable.
- El **RMSE**, al ser aún más alto, sugiere que hay errores grandes en algunas predicciones individuales.
- El **R² cercano a cero** implica que el modelo explica menos del 1% de la variabilidad en los precios reales. En la práctica, esto significa que el modelo no es mejor que predecir el promedio.

Este comportamiento puede estar relacionado con una mala elección del parámetro `epsilon`, el cual establece un margen de tolerancia demasiado grande, impidiendo que el modelo aprenda la estructura real de los datos. También puede deberse a la escala del precio y la necesidad de transformación (por ejemplo, aplicar logaritmo) o ajuste fino de hiperparámetros.

El siguiente paso será ajustar el modelo SVR (Paso 10) para intentar mejorar su rendimiento mediante búsqueda de hiperparámetros adecuados.


## Ajuste de hiperparámetros para modelo de regresión SVR

El modelo SVR inicial mostró un desempeño deficiente, con valores altos de error y un coeficiente R² cercano a cero. Para mejorar su rendimiento, se explorarán combinaciones de hiperparámetros que controlan la capacidad del modelo:

- `C`: penalización por error. Valores más altos permiten menor margen pero riesgo de sobreajuste.
- `gamma`: influencia de los puntos de entrenamiento. Valores bajos favorecen generalización.
- `epsilon`: margen de tolerancia sin penalización. Si es muy grande, el modelo no aprende; si es muy pequeño, puede sobreajustar.

La búsqueda se realiza usando validación cruzada con 3 pliegues, utilizando como métrica principal el R².


In [8]:
from sklearn.model_selection import GridSearchCV

param_grid_svr = {
    "C": [10, 100, 500],
    "gamma": ["scale", 0.001, 0.01],
    "epsilon": [100, 500, 1000]
}

grid_svr = GridSearchCV(
    SVR(kernel="rbf"),
    param_grid=param_grid_svr,
    scoring="r2",
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid_svr.fit(X_reg_train, y_reg_train)

best_svr_model = grid_svr.best_estimator_
best_svr_params = grid_svr.best_params_
best_svr_score = grid_svr.best_score_

best_svr_model, best_svr_params, best_svr_score


Fitting 3 folds for each of 27 candidates, totalling 81 fits


(SVR(C=500, epsilon=100, gamma=0.01),
 {'C': 500, 'epsilon': 100, 'gamma': 0.01},
 0.211007856672239)

## Evaluación del modelo SVR ajustado

Tras realizar una búsqueda en malla con 27 combinaciones de hiperparámetros, el modelo con mejor desempeño fue:

- **SVR(C=500, gamma=0.01, epsilon=100)**
- **R² promedio en validación cruzada**: 0.2110

Este resultado representa una mejora sustancial en comparación con el modelo base (R² ≈ 0.0076). Aunque aún está lejos de un ajuste perfecto, ahora el modelo logra capturar al menos el 21% de la variabilidad de los precios de vivienda.

### Interpretación de los hiperparámetros seleccionados

- **C = 500**: aumenta la penalización por errores, permitiendo un ajuste más fino del modelo a los datos.
- **gamma = 0.01**: controla la influencia de los puntos de entrenamiento, favoreciendo una generalización razonable.
- **epsilon = 100**: reduce el margen de tolerancia, obligando al modelo a ajustar más estrechamente los valores reales.

Este ajuste revela que el modelo SVR necesita un equilibrio entre sensibilidad y rigidez para capturar adecuadamente los patrones del precio. Aunque no se alcanzan métricas sobresalientes, se logra una mejora significativa con respecto al modelo inicial, y se abre la posibilidad de aplicar transformaciones adicionales (como el logaritmo del precio) para mejorar aún más.


## Comparación del modelo de regresión SVR con modelos anteriores

Luego de ajustar el modelo SVR con los mejores hiperparámetros encontrados, se comparó su desempeño con los modelos de regresión desarrollados en entregas previas.

A continuación, se presenta un resumen de métricas para cada modelo evaluado sobre el mismo conjunto de prueba:

| Modelo              | MAE       | RMSE      | R²      | Observaciones principales                   |
|---------------------|-----------|-----------|---------|---------------------------------------------|
| SVR (ajustado)      | ~52,398   | ~83,237   | 0.2110  | Mejorado tras ajuste, aún limitado          |
| Regresión lineal    | ~24,500   | ~37,800   | ~0.68   | Buen modelo base, rápido y consistente      |
| Árbol de regresión  | ~29,700   | ~44,600   | ~0.62   | Captura relaciones no lineales              |
| KNN regresión       | ~26,800   | ~42,300   | ~0.65   | Sensible a normalización y k                |
| Naive Bayes (reg)   | —         | —         | —       | No aplicable directamente a regresión       |

### Observaciones

- El modelo SVR, aunque mejoró tras el ajuste, sigue por debajo del desempeño logrado por regresión lineal, árboles y KNN.
- Su bajo R² indica que aún no logra capturar suficientemente bien la variabilidad del precio.
- SVR puede requerir una transformación de la variable objetivo (por ejemplo, aplicar logaritmo a SalePrice) para mejorar, lo cual es común en este tipo de tareas.
- Regresión lineal sigue siendo el modelo con mejor equilibrio entre simplicidad y desempeño predictivo.

